# 데코레이터의 활용 - 흔한 실수 피하기

## 래핑된 원본 객체의 데이터 보존
데코레이터 사용 시 많이 발생하는 문제 중 하나는 원본 함수의 일부 프로퍼티 또는 속성을 유지하지 않아 원하지 않는 부작용을 유발한다는 것  

아래 코드는 함수가 실행될 때 로그를 남기는 데코레이터

In [8]:
# decorator_wraps_1.py
def trace_decorator(function):
    def wrapped(*args, **kwargs):
        logger.info("%s 실행", function.__qualname__)
        return function(*args, **kwargs)
    
    return wrapped

In [9]:
@trace_decorator
def process_account(account_id):
    """id별 계정 처리"""
    logger.info("%s 계정 처리", account_id)

In [10]:
help(process_account)

Help on function wrapped in module __main__:

wrapped(*args, **kwargs)



In [11]:
print(process_account.__qualname__)

trace_decorator.<locals>.wrapped


데코레이터는 원래 함수의 어떤 것도 변경하지 않아야 하지만 코드에 결함이 있어서 이름이나 docstring을 변경하는 경우가 있음  
위의 경우에도 데코레이터가 실제로 원봄 함수를 wrapped라 불리는 새로운 함수로 변경했기 때문에 원본 함수의 이름이 아닌 새로운 함수의 이름을 출력하게 됨 

위의 데코레이터를 이름이 다른 여러 함수에 적용하더라고 결국은 wrapped라는 이름만 출력하게 됨  
-> 개별 함수를 확인하고 싶은 경우에 실제 실행된 함수를 알 수 없으므로 오히려 디버깅이 더 어려워짐

이런 문제를 수정하기 위해서는 래핑된 함수, 즉 wrapped 함수에 **@wraps** 데코레이터를 적용하여 실제로는 fucntion 파리미터 함수를 래핑한 것이라고 알려주는 것

In [13]:
# decorator_wraps_2.py

from functools import wraps

def trace_decorator(function):
    @wraps(function)
    def wrapped(*args, **kwargs):
        logger.info("running %s", function.__qualname__)
        return function(*args, **kwargs)
    
    return wrapped

@trace_decorator
def process_account(account_id):
    """id별 계정 처리"""
    logger.info("%s 계정 처리", account_id)

In [14]:
help(process_account)

Help on function process_account in module __main__:

process_account(account_id)
    id별 계정 처리



In [15]:
process_account.__qualname__

'process_account'

이와 같이 @wraps 데코레이터를 사용한 결과 원본 함수에 대한 정보를 얻을 수 있게 됨  

또한 @wraps 데코레이터를 사용하면 docstring에 포함된 단위 테스트 기능이 복구됨  
@wraps 데코레이터를 사용해 &#95;&#95;wrapped&#95;&#95; 속성을 통해 수정되지 않은 원본에도 접근할 수 있게 됨.

In [16]:
def decorator(original_function):
    @wraps(original_function)
    def decorated_function(*args, **kwargs):
        # 데코레이터에 의한 수정 작업 ...
        return original_function(*args, **kwargs)
    
    return decorated_function